# SPECTRE TEST
- Using H5 Model trained using ISCX-IDS-2012 Dataset
- Testing using Malware_Capture_Facility_Project Dataset - CTU-Malware-Capture-Botnet-135-1 Stlrat DDoS (MD5: `c7838b75ba10b0341554d25fbcc3bbc0`)

---

## Import Dependencies

In [12]:
import sys
from scapy.all import *
import h5py
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from tabulate import tabulate

import seaborn as sns # Graphing, built ontop of MatPlot for ease-of-use and nicer diagrams.
import sklearn # For Machine Learning algorithms
import scikitplot # Confusion matrix plotting
from sklearn.decomposition import PCA # For PCA dimensionality reduction technique
from sklearn.preprocessing import StandardScaler # For scaling to unit scale, before PCA application
from sklearn.preprocessing import LabelBinarizer # For converting categorical data into numeric, for modeling stage
from scikitplot.metrics import plot_confusion_matrix # For plotting confusion matrices
from sklearn.metrics import accuracy_score # For getting the accuracy of a model's predictions
from sklearn.metrics import classification_report # Various metrics for model performance
from sklearn.model_selection import StratifiedKFold # For optimal train_test splitting, for model input data


In [ ]:
# KAFKA IMPORT

## METHODS

# Import Data

## Load the npy file

In [3]:
# ISCX-IDS-2012 - 14 JUN
#mal_pcap = scapy.rdpcap('/mnt/Data/SPECTRE/Dataset/ISCX-2012/PCAP/testbed-14jun.pcap') 
#training_sample = np.load('/home/aryn/spectre-dev/dataset/ISCX-IDS-2012/PCAP-NPY/destinationPayload_TestbedMonJun14Flows.xml.npy', allow_pickle=True)

# Malware_Capture_Facility_Project - CTU-Malware-Capture-Botnet-135-1 Stlrat DDoS
# malicious_data = np.load('/home/aryn/spectre-dev/dataset/Malware_Capture_Facility_Project/PCAP-NPY/2015-09-10_winlinux.npy', allow_pickle=True)

In [ ]:
rows_mal = malicious_data[:2]
table_mal = rows_mal.tolist()
print(tabulate(table_mal))

In [4]:
rows_train = training_sample[:2]
table_train = rows_train.tolist()
print(tabulate(table_train))

-----------------------------  -
[ 43  79  75 ... 103 101 100]  0
[ 46  46  46 ... 110  46  46]  0
-----------------------------  -


/home/aryn/miniconda3/envs/spectre/lib/python3.9/site-packages/tabulate/__init__.py:107: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  (len(row) >= 1 and row[0] == SEPARATING_LINE)


## Load the csv file

In [2]:
def clean_dataset(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep]

In [3]:
def get_PCA_feature_names(num_of_pca_components):
    feature_names = []
    for i in range(num_of_pca_components):    
        feature_names.append(f"Principal component {i+1}")
    return feature_names

**Useful environment variables**

In [4]:
# 'Reduced dimensions' variable for altering the number of PCA principal components. Can be altered for needs.
# Only 7 principal components needed when using non-normalised PCA dataset.
dimensions_num_for_PCA = 7

# Max number of permutations to run. Can be altered for needs.
number_of_permutations = 100

# 10 folds is usually the heuristic to follow for larger datasets of around this size.
num_of_splits_for_skf = 10

# Seed value to pass into models so that repeated runs result in the same output
seed_val = 1

# Number of statistical distance measures to run (for the results, columns section)
num_of_statistical_dist_measures = 6

### Import Data

In [7]:
Normal_Data = pd.read_csv('../../dataset/CICIDS2017/MachineLearningCSV/MachineLearningCVE/Monday-WorkingHours.pcap_ISCX.csv')
df = Normal_Data.copy()

### Prepare the data for the model

In [8]:
# Renaming columns and creating a copy
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
df_cleaned = df.copy()
df_cleaned = clean_dataset(df_cleaned)

# Resetting index and removing unneeded index column
df_cleaned = df_cleaned.reset_index()
df_cleaned.drop('index', axis=1, inplace=True)

# Saving the label attribute before dropping it
df_labels = df_cleaned['label']
df_no_labels = df_cleaned.drop('label', axis=1, inplace=False)
df_features = df_no_labels.columns.tolist()

# Scaling the data
df_scaled = StandardScaler().fit_transform(df_no_labels)
df_scaled = pd.DataFrame(data=df_scaled, columns=df_features)

# Performing PCA
pca = PCA(n_components=dimensions_num_for_PCA)
principal_components = pca.fit(df_no_labels).transform(df_no_labels)

# Creating a DataFrame with principal components
principal_component_headings = get_PCA_feature_names(dimensions_num_for_PCA)
df_pc = pd.DataFrame(data=principal_components, columns=principal_component_headings)

# Concatenating principal components and labels
df_final = pd.concat([df_pc, df_labels], axis=1)

# Applying LabelBinarizer to the labels
lb = LabelBinarizer()
df_final['label'] = lb.fit_transform(df_final['label'])

# Displaying the final DataFrame
df_final

/tmp/ipykernel_588891/2188718236.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
/tmp/ipykernel_588891/2188718236.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
/tmp/ipykernel_588891/2671744760.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
  indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)


,Principal component 1,Principal component 2,Principal component 3,Principal component 4,Principal component 5,Principal component 6,Principal component 7,label
0,-1.983320e+07,9.251362e+05,1.046049e+06,-128074.061738,-869081.641179,-9328.258030,10210.389224,0
1,-1.994976e+07,1.872796e+06,1.000291e+07,-124409.821468,-846672.396250,-10321.565566,10014.928220,0
2,-1.994976e+07,1.872796e+06,1.000291e+07,-124409.821468,-846672.396250,-10321.565566,10014.928220,0
3,-1.994976e+07,1.872796e+06,1.000291e+07,-124409.821468,-846672.396250,-10321.565566,10014.928220,0
4,-1.984615e+07,1.030431e+06,2.041257e+06,-127666.256450,-866593.042249,-9438.611625,10188.446052,0
...,...,...,...,...,...,...,...,...
529476,-1.978012e+07,6.112687e+05,-1.939024e+06,-133315.868223,-855223.925380,-9350.481582,16850.957232,0
529477,-1.974771e+07,6.161432e+05,-1.935949e+06,-128750.407703,-841842.148056,-9094.307239,10005.865241,0
529478,-1.980753e+07,7.185561e+05,-9.063745e+05,-128879.080536,-873765.673193,-9182.189422,10266.382651,0
529479,-1.981276e+07,7.612775e+05,-5.024661e+05,-128678.971810,-872603.283762,-9206.582296,10228.529420,0


In [13]:
X = df_final.drop(['label'], axis = 1)
y = df_final['label']
y

0         0
1         0
2         0
3         0
4         0
         ..
529476    0
529477    0
529478    0
529479    0
529480    0
Name: label, Length: 529481, dtype: int64

In [14]:
skf = StratifiedKFold(n_splits=num_of_splits_for_skf, shuffle=False)
skf

StratifiedKFold(n_splits=10, random_state=None, shuffle=False)

In [15]:
import h5py

# Assuming preprocessed_data is a NumPy array containing the preprocessed data
with h5py.File('preprocessed_data.h5', 'w') as f:
    f.create_dataset('data', data=skf)

TypeError: Object dtype dtype('O') has no native HDF5 equivalent

# Predection

## Model Preperation

In [ ]:
# Load TF Saved_model
spectre_model = tf.saved_model.load("/home/aryn/spectre-dev/spectre-code/spectre-ann/Model/DDOS/SavedModel")

### Load H5 Model

In [6]:
# H5 Model
spectre_model_h5 = keras.models.load_model('/home/aryn/spectre-dev/spectre-code/spectre-ann/Model/DDOS/spectre_ddos_h5.h5')

2023-05-09 03:34:13.057092: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2023-05-09 03:34:13.057126: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: aryn-eos
2023-05-09 03:34:13.057134: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: aryn-eos
2023-05-09 03:34:13.057293: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 530.41.3
2023-05-09 03:34:13.057315: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 530.41.3
2023-05-09 03:34:13.057321: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:309] kernel version seems to match DSO: 530.41.3


In [10]:
# H5 Model 2
spectre_model_2_h5 = keras.models.load_model('../spectre-ann/Model/DDOS_2/spectre_ddos_2_h5.h5')

### Load tflite Model

In [ ]:
# tflite Model
# Load the TFLite model in TFLite Interpreter
#interpreter = tf.lite.Interpreter('/home/aryn/spectre-dev/spectre-code/spectre-ann/Model/DDOS/spectre_ddos_lite.tflite')
#interpreter.allocate_tensors()

# Get input and output tensors.
#input_details = interpreter.get_input_details()
#output_details = interpreter.get_output_details()

In [ ]:
#try:
    # Load the H5 model
#    with h5py.File('/home/aryn/spectre-dev/spectre-code/spectre-ann/Model/DDOS/FYP_Finalh5.h5', 'r') as spectre_model:
        # Print metadata
#        print("H5 file metadata:")
#        print("==================")
#        for key, value in spectre_model.attrs.items():
#            print(f"{key}: {value}")
#        print("==================")
#        print("Done!")
#except:
#    print("Error loading H5 file.")

## Model Prediction

### Use the model to predict anomalies

In [ ]:
#model = spectre_model

In [ ]:
#predictions = spectre_model_2_h5.predict(training_sample)

In [ ]:
import h5py

# Assuming preprocessed_data is a NumPy array containing the preprocessed data
with h5py.File('preprocessed_data.h5', 'w') as f:
    f.create_dataset('data', data=preprocessed_data)

## Results

In [ ]:
x = ['Detection rate', 'Anomalies detected']
y = [detection_rate, anomalies_detected]

plt.bar(x, y)
plt.show()